# TASSONOMIA SARS CoV-2


In [2]:
#!pip install biopython
#!pip install entrezpy
from Bio import Entrez
import pandas as pd
import numpy as np
import os

Entrez.email = "a.fossati@campus.unimib.it"
data = Entrez.efetch(db='taxonomy', id='txid2697049', retmode='xml')
record = Entrez.read(data)
print(data , "\n")
print("Keys are:",record[0].keys(), "\n")
print("Lineage is:", record[0]['Lineage'], "\n") # interrogazione chiavi del dizionario
print("LineageEx is:", record[0]['LineageEx'], "\n") 

print("Division:",record[0]['Division'], "\n")
print("ScientificName:",record[0]['ScientificName'], "\n")
print("OtherNames:",record[0]['OtherNames'], "\n")
print("ParentTaxId:",record[0]['ParentTaxId'], "\n")



Keys are: dict_keys(['TaxId', 'ScientificName', 'OtherNames', 'ParentTaxId', 'Rank', 'Division', 'GeneticCode', 'MitoGeneticCode', 'Lineage', 'LineageEx', 'CreateDate', 'UpdateDate', 'PubDate']) 

Lineage is: Viruses; Riboviria; Orthornavirae; Pisuviricota; Pisoniviricetes; Nidovirales; Cornidovirineae; Coronaviridae; Orthocoronavirinae; Betacoronavirus; Sarbecovirus; Severe acute respiratory syndrome-related coronavirus 

LineageEx is: [{'TaxId': '10239', 'ScientificName': 'Viruses', 'Rank': 'superkingdom'}, {'TaxId': '2559587', 'ScientificName': 'Riboviria', 'Rank': 'clade'}, {'TaxId': '2732396', 'ScientificName': 'Orthornavirae', 'Rank': 'kingdom'}, {'TaxId': '2732408', 'ScientificName': 'Pisuviricota', 'Rank': 'phylum'}, {'TaxId': '2732506', 'ScientificName': 'Pisoniviricetes', 'Rank': 'class'}, {'TaxId': '76804', 'ScientificName': 'Nidovirales', 'Rank': 'order'}, {'TaxId': '2499399', 'ScientificName': 'Cornidovirineae', 'Rank': 'suborder'}, {'TaxId': '11118', 'ScientificName': 'C

In [ ]:
#LAVORO COVID CON AFFILIATION INFO

In [1]:
#!pip install biopython
#!pip install entrezpy
from Bio import Entrez
import pandas as pd
import numpy as np
import os

Entrez.email = "l.pretini@campus.unimib.it"

In [37]:
#txid2697049 = sars-cov-2
search = Entrez.esearch(db="pubmed", term='Sars-Cov 2', retmax=80000, mindate=2019/12/20) #"txid2697049[Organism:exp]"
search2 = Entrez.read(search)
covid_idlist=search2['IdList']
#remove duplicates ids
li_id_covid=list(dict.fromkeys(covid_idlist))
len(li_id_covid)  #numero di pubblicazioni totali sul covid, che poi divranno essere studiate per paese


77158

In [87]:
from urllib.error import HTTPError
import time
country_cov=[]
for ids in li_id_covid[42223:77157]:
    listine=[]
    try:
      hInfo = Entrez.efetch(db="pubmed", id = ids, retmode= 'xml') 
      ent= Entrez.read(hInfo)
      affil_cov=ent['PubmedArticle'][0]['MedlineCitation']['Article']['AuthorList']
      count=-1
      for i in affil_cov:
        count+=1
        if len(affil_cov[count]['AffiliationInfo'])!=0:
            listine.append(affil_cov[count]['AffiliationInfo'][0]['Affiliation'])
      listine.append(count+1)
      country_cov.append(listine)
    except KeyError:
      country_cov.append(['no_aff'])  
    except IndexError:
      country_cov.append(['no_aff'])
    except TimeoutError:
      time.sleep(800)
    except HTTPError as e: 
      if e.code == 429:
        time.sleep(1500)
      if e.code == 400:
        country_cov.append(['bad_request'])
        time.sleep(100)
    except ConnectionAbortedError:
      time.sleep(800)
len(country_cov)




34932

In [99]:
df_covid_aff=pd.DataFrame(list(zip(li_id_covid,country_cov)), columns=['id','Affiliations'])
affil_covid_def=df_covid_aff.reset_index()
affil_covid_def=affil_covid_def.drop('index',axis=1)
affil_covid_def

,id,Affiliations
0,33906236,"[Institute of Virology, Medical Faculty, Unive..."
1,33906227,"[Brigham and Women's Hospital, Harvard Medical..."
2,33906143,[Servicio de Prevención y Control de Infeccion...
3,33906142,[Sección Fisiopatología Respiratoria y Laborat...
4,33906130,"[Unidad Febril de Urgencias, Hospital de Infec..."
...,...,...
77149,33665649,"[Institute of Medical Virology, Justus Liebig ..."
77150,32655062,"[Department of Cell Biology, The Johns Hopkins..."
77151,30918070,[2]
77152,30867314,[2]


In [100]:
affil_covid_def.to_csv('covid_affiliations.csv') #export

In [ ]:
#seconda parte di raccolta riguarda abstract e data

In [ ]:
#cattura abstract e data per covid

In [2]:
cov=pd.read_csv("C:/Users/aless/OneDrive/Desktop/AleBicocca/Data Science/Anno 2/Lab Biosciences/Project/Dati_Covid/covid_affiliations.csv",converters={'Affiliations': eval})
li_id_covid=cov['id'].values.tolist()
len(li_id_covid)

77154

In [3]:
from urllib.error import HTTPError
import time

abstr=[]
date=[]

for ids in li_id_covid:
    i=0
    
    try:
      hInfo = Entrez.efetch(db="pubmed", id = ids, retmode= 'xml') 
      ent= Entrez.read(hInfo)
    except TimeoutError:
      time.sleep(800)
    except HTTPError as e: 
      if e.code == 429:
        time.sleep(1000)
      if e.code == 400:
        abstr.append(['bad_request'])
        date.append(['bad_request'])
        i+=1
        time.sleep(100)
    except ConnectionAbortedError:
      time.sleep(800)
    
    if i==0: #variabile flag per bad request
        try:
          abstract= ent['PubmedArticle'][0]['MedlineCitation']['Article']['Abstract']['AbstractText'][0]
          abstr.append(abstract)
        except KeyError:
          abstr.append(['no_abstr'])  
        except IndexError:
          abstr.append(['no_abstr'])
        except TimeoutError:
          time.sleep(800)
        except HTTPError as e: 
          if e.code == 429:
            time.sleep(1000)
          if e.code == 400:
            abstr.append(['bad_request'])
            time.sleep(100)
        except ConnectionAbortedError:
          time.sleep(800)
    
    if i==0:
        try:
          data=ent['PubmedArticle'][0]['PubmedData']['History']
          date.append(data)
        except KeyError:
          date.append(['no_date'])  
        except IndexError:
          date.append(['no_date'])
        except TimeoutError:
          time.sleep(800)
        except HTTPError as e: 
          if e.code == 429:
            time.sleep(1000)
          if e.code == 400:
            date.append(['bad_request'])
            time.sleep(100)
        except ConnectionAbortedError:
          time.sleep(800)
    
print(len(abstr))
print(len(date))


77154
77154


In [4]:
cov['Abstract']=abstr
cov['Date']=date
cov=cov.drop('Unnamed: 0',axis=1)
cov.to_csv('covid_completo.csv')

In [5]:
cov

,id,Affiliations,Abstract,Date
0,33906236,"[Institute of Virology, Medical Faculty, Unive...",The SARS-CoV-2 pandemic has led to the develop...,"[{'Year': '2021', 'Month': '03', 'Day': '03'},..."
1,33906227,"[Brigham and Women's Hospital, Harvard Medical...",Both SARS-CoV-2 reinfection and persistent inf...,"[{'Year': '2021', 'Month': '02', 'Day': '26'},..."
2,33906143,[Servicio de Prevención y Control de Infeccion...,The COVID-19 pandemic has had an impact on pub...,"[{'Year': '2021', 'Month': '4', 'Day': '27', '..."
3,33906142,[Sección Fisiopatología Respiratoria y Laborat...,The airborne spread is the main route of human...,"[{'Year': '2021', 'Month': '4', 'Day': '27', '..."
4,33906130,"[Unidad Febril de Urgencias, Hospital de Infec...",Febrile Emergency Units were annexed to tertia...,"[{'Year': '2021', 'Month': '4', 'Day': '27', '..."
...,...,...,...,...
77149,33665649,"[Institute of Medical Virology, Justus Liebig ...",Hospitals and rehabilitation centers around th...,"[{'Year': '2021', 'Month': '3', 'Day': '5', 'H..."
77150,32655062,"[Department of Cell Biology, The Johns Hopkins...",[no_abstr],"[{'Year': '2020', 'Month': '7', 'Day': '14', '..."
77151,30918070,[2],Coronavirus nonstructural protein 8 (nsp8) has...,"[{'Year': '2019', 'Month': '02', 'Day': '19'},..."
77152,30867314,[2],Coronaviruses (CoVs) assemble by budding into ...,"[{'Year': '2019', 'Month': '01', 'Day': '03'},..."


In [57]:
ind=[]
for index,row in cov.iterrows():
    try:
        if str(row['Date'][0]['Year']) != 2020 or str(row['Date'][0]['Year']) != 2021:
            ind.append(index)
    except KeyError:
        ind.append(index)
    except TypeError:
        ind.append(index)
len(ind)

77154

In [53]:
type(str(cov['Date'][0][0]['Year']))

str

In [39]:
cov['Date'][0:77151][0]

[DictElement({'Year': '2021', 'Month': '03', 'Day': '03'}, attributes={'PubStatus': 'received'}), DictElement({'Year': '2021', 'Month': '4', 'Day': '27', 'Hour': '20', 'Minute': '27'}, attributes={'PubStatus': 'entrez'}), DictElement({'Year': '2021', 'Month': '4', 'Day': '28', 'Hour': '6', 'Minute': '0'}, attributes={'PubStatus': 'pubmed'}), DictElement({'Year': '2021', 'Month': '4', 'Day': '28', 'Hour': '6', 'Minute': '0'}, attributes={'PubStatus': 'medline'})]

In [ ]:
#VICINI COVID RACCOLTA DATI PUBBLICAZIONI

In [1]:
#!pip install biopython
#!pip install entrezpy
from Bio import Entrez
import pandas as pd
import numpy as np
import os

Entrez.email = "l.pretini@campus.unimib.it"

In [2]:
from ete3 import NCBITaxa
ncbi = NCBITaxa()
ncbi.update_taxonomy_database()



NCBI database not present yet (first time used?)
Done. Parsing...


Loading node names...
2324161 names loaded.
245035 synonyms loaded.
Loading nodes...
2324161 nodes loaded.
Linking nodes...
Tree is loaded.
Updating database: /.etetoolkit\taxa.sqlite ...
 2324000 generating entries...  generating entries... generating entries...  generating entries... generating entries...  generating entries... 

Inserting synonyms:      10000 


Uploading to /.etetoolkit\taxa.sqlite



Inserting taxid merges:  15000  

...

Loading node names...
2324161 names loaded.
245035 synonyms loaded.
Loading nodes...
2324161 nodes loaded.
Linking nodes...
Tree is loaded.
Updating database: /.etetoolkit\taxa.sqlite ...
 2324000 generating entries... 33000 generating entries...  generating entries...  626000 generating entries... generating entries...  generating entries... 1050000 generating entries...    generating entries...   
Uploading to /.etetoolkit\taxa.sqlite


Inserting synonyms:      10000 

Inserting taxid merges:  20000  

Inserting taxids:       10000  

Inserting taxids:       2320000   

In [ ]:
#lista dalla quale togliamo il covid
desc = ncbi.get_descendant_taxa('Coronaviridae')
descendants=ncbi.translate_to_names(desc)
print(len(descendants))
print(descendants)

In [9]:
#txid2697049 = sars-cov-2
parenti_idlist=[]
for parente in descendants:
    search = Entrez.esearch(db="pubmed", term=parente, retmax=20000, maxdate=2019/12/20) #"txid2697049[Organism:exp]"
    search2 = Entrez.read(search)
    idlist=search2['IdList']
    parenti_idlist.append(idlist)



TypeError: unhashable type: 'ListElement'

In [12]:
vicini_idlist=[]
for lista in parenti_idlist:
    for index in lista:
        vicini_idlist.append(index)
len(vicini_idlist)
vicini_idlist=list(dict.fromkeys(vicini_idlist))
len(vicini_idlist)  #100541 contro 75k del covid

100541

In [ ]:
# NUOVO 
#se funziona, prendi anche date covid

In [15]:
from urllib.error import HTTPError
import time

country_par=[]
dates=[]

for ids in vicini_idlist:
    listine=[]
    
    try:
      hInfo = Entrez.efetch(db="pubmed", id = ids, retmode= 'xml') 
      ent= Entrez.read(hInfo)
    except TimeoutError:
      time.sleep(800)
    except HTTPError as e: 
      if e.code == 429:
        time.sleep(1500)
      if e.code == 400:
        country_par.append(['bad_request'])
        time.sleep(100)
    except ConnectionAbortedError:
      time.sleep(800)
    
    
    try:
      affil_cov=ent['PubmedArticle'][0]['MedlineCitation']['Article']['AuthorList']
      count=-1
      for i in affil_cov:
        count+=1
        if len(affil_cov[count]['AffiliationInfo'])!=0:
            listine.append(affil_cov[count]['AffiliationInfo'][0]['Affiliation'])
      listine.append(count+1)
      country_par.append(listine)
    except KeyError:
      country_par.append(['no_aff'])  
    except IndexError:
      country_par.append(['no_aff'])
    except TimeoutError:
      time.sleep(800)
    except HTTPError as e: 
      if e.code == 429:
        time.sleep(1500)
      if e.code == 400:
        country_par.append(['bad_request'])
        time.sleep(100)
    except ConnectionAbortedError:
      time.sleep(800)
    
    try:
      data=ent['PubmedArticle'][0]['PubmedData']['History']
      dates.append(data)
    except KeyError:
      dates.append(['no_date'])  
    except IndexError:
      dates.append(['no_date'])
    except TimeoutError:
      time.sleep(800)
    except HTTPError as e: 
      if e.code == 429:
        time.sleep(1500)
      if e.code == 400:
        country_par.append(['bad_request'])
        time.sleep(100)
    except ConnectionAbortedError:
      time.sleep(800)
    
print(len(country_par))
print(len(dates))


100542
100541


In [16]:
df_parenti_aff_dates=pd.DataFrame(list(zip(vicini_idlist,country_par,dates)), columns=['id','Affiliations','dates'])
affildates_par_def=df_parenti_aff_dates.reset_index()
affildates_par_def=affildates_par_def.drop('index',axis=1)
affildates_par_def

,id,Affiliations,dates
0,33912590,[Beijing Advanced Innovation Centre for Biomed...,"[{'Year': '2020', 'Month': '12', 'Day': '30'},..."
1,33899350,"[Finnoflag Oy, Kuopio, Finland., 1]","[{'Year': '2021', 'Month': '4', 'Day': '26', '..."
2,33895786,[Institute for Interdisciplinary Information S...,"[{'Year': '2020', 'Month': '09', 'Day': '26'},..."
3,33888900,[NHC Key Laboratory of Enteric Pathogenic Micr...,"[{'Year': '2020', 'Month': '12', 'Day': '28'},..."
4,33869323,"[Center for Food Animal Health Research, Colle...","[{'Year': '2020', 'Month': '12', 'Day': '17'},..."
...,...,...,...
100536,27309961,[12],"[{'Year': '2016', 'Month': '02', 'Day': '19'},..."
100537,19110700,"[Department of Microbiology, College of Medici...","[{'Year': '2008', 'Month': '12', 'Day': '30', ..."
100538,32994418,[Chicago Zoological and Aquatic Animal Residen...,"[{'Year': '2020', 'Month': '06', 'Day': '10'},..."
100539,32701249,[National Kunming High-Level Biosafety Primate...,"[{'Year': '2020', 'Month': '7', 'Day': '24', '..."


In [18]:
affildates_par_def.to_csv("affildates_par_def.csv")

In [ ]:
#per vicini covid c'è da raccolgiere nuovamente data e abstract, come fatto per covid, conservando invece id e affiliazione

In [ ]:
parenti=pd.read_csv("C:/Users/aless/OneDrive/Desktop/AleBicocca/Data Science/Anno 2/Lab Biosciences/Project/Dati_Covid/affildates_par_def.csv",converters={'Affiliations': eval})
parenti=parenti.drop('dates',axis=1)
vicini_id_list=parenti['id'].values.tolist()
len(vicini_id_list)

In [ ]:
from urllib.error import HTTPError
import time

abstr=[]
date=[]

for ids in vicini_id_list:
    i=0
    
    try:
      hInfo = Entrez.efetch(db="pubmed", id = ids, retmode= 'xml') 
      ent= Entrez.read(hInfo)
    except TimeoutError:
      time.sleep(800)
    except HTTPError as e: 
      if e.code == 429:
        time.sleep(1500)
      if e.code == 400:
        abstr.append(['bad_request'])
        date.append(['bad_request'])
        i+=1
        time.sleep(100)
    except ConnectionAbortedError:
      time.sleep(800)
    
    if i==0: #variabile flag per bad request
        try:
          abstract= ent['PubmedArticle'][0]['MedlineCitation']['Article']['Abstract']['AbstractText'][0]
          abstr.append(abstract)
        except KeyError:
          abstr.append(['no_abstr'])  
        except IndexError:
          abstr.append(['no_abstr'])
        except TimeoutError:
          time.sleep(800)
        except HTTPError as e: 
          if e.code == 429:
            time.sleep(1500)
          if e.code == 400:
            abstr.append(['bad_request'])
            time.sleep(100)
        except ConnectionAbortedError:
          time.sleep(800)
    
    if i==0:
        try:
          data=ent['PubmedArticle'][0]['PubmedData']['History']
          date.append(data)
        except KeyError:
          date.append(['no_date'])  
        except IndexError:
          date.append(['no_date'])
        except TimeoutError:
          time.sleep(800)
        except HTTPError as e: 
          if e.code == 429:
            time.sleep(1500)
          if e.code == 400:
            date.append(['bad_request'])
            time.sleep(100)
        except ConnectionAbortedError:
          time.sleep(800)
    
print(len(abstr))
print(len(date))


In [ ]:
parenti['Abstract']=abstr
parenti['Data']=date
parenti=parenti.drop('Unnamed: 0',axis=1)
parenti.to_csv('parenti_completo.csv')

In [ ]:
parenti

In [ ]:
#EFFETTUA LAVORO GIORNALI ANCHE PER VICINI 

In [ ]:
#VICINI COVID

In [24]:
#txid2697049 = sars-cov-2
anno=['1970[pdat]','1971[pdat]','1972[pdat]','1973[pdat]','1974[pdat]','1975[pdat]','1976[pdat]','1977[pdat]','1978[pdat]','1979[pdat]','1980[pdat]','1981[pdat]','1982[pdat]','1983[pdat]','1984[pdat]','1985[pdat]','1986[pdat]','1987[pdat]','1988[pdat]','1989[pdat]','1990[pdat]','1991[pdat]','1992[pdat]','1993[pdat]','1994[pdat]','1995[pdat]','1996[pdat]','1997[pdat]','1998[pdat]','1999[pdat]','2000[pdat]','2001[pdat]','2002[pdat]','2003[pdat]','2004[pdat]','2005[pdat]','2006[pdat]','2007[pdat]','2008[pdat]','2009[pdat]','2010[pdat]','2011[pdat]','2012[pdat]','2013[pdat]','2014[pdat]','2015[pdat]','2016[pdat]','2017[pdat]','2018[pdat]','2019[pdat]']
parenti_idlist=[]
for el in anno:
    search = Entrez.esearch(db="pubmed", term='Coronavirus and '+str(el), retmax=100000, datetype='pdat') #"txid2697049[Organism:exp]"
    search2 = Entrez.read(search)
    for ids in search2['IdList']:
        parenti_idlist.append(ids)
#remove duplicates ids
li_id_parenti=list(dict.fromkeys(parenti_idlist))
len(li_id_parenti)  #numero di pubblicazioni totali sul covid, che poi divranno essere studiate per paese

14703

In [27]:
from urllib.error import HTTPError
import time

country_par=[]
date=[]

for ids in li_id_parenti:
    i=0
    listine=[]
    try:
      hInfo = Entrez.efetch(db="pubmed", id = ids, retmode= 'xml') 
      ent= Entrez.read(hInfo)
    except TimeoutError:
      time.sleep(800)
    except HTTPError as e: 
      if e.code == 429:
        time.sleep(1000)
      if e.code == 400:
        country_par.append(['bad_request'])
        date.append(['bad_request'])
        i+=1
        time.sleep(100)
    except ConnectionAbortedError:
      time.sleep(800)
    
    if i==0: #variabile flag per bad request
        try:
            affil_cov=ent['PubmedArticle'][0]['MedlineCitation']['Article']['AuthorList']
            count=-1
            for i in affil_cov:
                count+=1
                if len(affil_cov[count]['AffiliationInfo'])!=0:
                    listine.append(affil_cov[count]['AffiliationInfo'][0]['Affiliation'])
            listine.append(count+1)
            country_par.append(listine)
        except KeyError:
          country_par.append(['no_aff'])  
        except IndexError:
          country_par.append(['no_aff'])
        except TimeoutError:
          time.sleep(800)
        except HTTPError as e: 
          if e.code == 429:
            time.sleep(1000)
          if e.code == 400:
            country_par.append(['bad_request'])
            time.sleep(100)
        except ConnectionAbortedError:
          time.sleep(800)
    
    if i==0:
        try:
          data=ent['PubmedArticle'][0]['PubmedData']['History']
          date.append(data)
        except KeyError:
          date.append(['no_date'])  
        except IndexError:
          date.append(['no_date'])
        except TimeoutError:
          time.sleep(800)
        except HTTPError as e: 
          if e.code == 429:
            time.sleep(1000)
          if e.code == 400:
            date.append(['bad_request'])
            time.sleep(100)
        except ConnectionAbortedError:
          time.sleep(800)
    
print(len(country_par))
print(len(date))

14703
102


In [66]:
df_parenti_aff_dates=pd.DataFrame(list(zip(li_id_parenti,country_par)), columns=['id','Affiliations'])
affildates_par_def=df_parenti_aff_dates.reset_index()
affildates_par_def=affildates_par_def.drop('index',axis=1)
affildates_par_def

,id,Affiliations
0,4991400,[3]
1,4991398,[4]
2,4323585,[2]
3,4323489,[2]
4,5504709,[2]
...,...,...
14698,29956635,[Department of Biochemistry and Molecular Biol...
14699,29905769,"[Department of Health-Abu Dhabi, United Arab E..."
14700,29792154,"[Centre for Nano and Material Sciences, Jain U..."
14701,29538674,"[Department of Pediatrics, Division of Infecti..."


In [67]:
affildates_par_def.to_csv('vicini_dateaff.csv')